In [1]:
using FFTW
using LinearAlgebra
using JLD2
using YAML
using PyPlot


config = YAML.load_file("../config.yaml")

Dict{Any, Any} with 16 entries:
  "DM_particle_mass_Dark" => 0.00472716
  "run_name"              => "density_cube_snap_099_full_dm"
  "xmin"                  => 0.0
  "num_scales"            => 10
  "GAS_particle_mass_TNG" => 0.000743736
  "output_directory"      => "/global/cscratch1/sd/james12/NEXUS_analysis/"
  "nx"                    => 1024
  "N_GAS"                 => 15625000000
  "num_snaps"             => 1
  "snapfile_root"         => 99
  "algorithm"             => "NEXUS+"
  "input_directory"       => "/global/cscratch1/sd/james12/Illustris_TNG_Data/T…
  "simulation_type"       => "Dark"
  "DM_particle_mass_TNG"  => 0.00398343
  "N_DM"                  => 15625000000
  "xmax"                  => 205000.0

In [2]:
using CosmoMMF

In [3]:
CosmoMMF.test_print() #Nice, the CosmoMMF Package imported properly

This test function works!

In [4]:
config["input_directory"]

# "/global/cscratch1/sd/james12/NEXUS_analysis/density_cube_snap_099_full_dm.jld2"

"/global/cscratch1/sd/james12/Illustris_TNG_Data/TNG_Density_Cubes/"

In [4]:
load_string = config["input_directory"] * config["run_name"] * ".jld2"

@load load_string den

print("The size of the original density cube:")
print(size(den))

The size of the original density cube:(1024, 1024, 1024)

In [5]:
den_256_subset = den[1:256,1:256,1:256]
den_512_subset = den[1:512,1:512,1:512]

print("We now want to test and see if hessian_fast and hessian_slow yield the same results on the subsets")

We now want to test and see if hessian_fast and hessian_slow yield the same results on the subsets

### 256^3 Hessian Calculation

In [6]:
n = 256
wave_vecs = CosmoMMF.wavevectors3D(Float64,(n,n,n))
density_field = den_256_subset .+ 0.0001 
@time smoothed_256 = CosmoMMF.smooth_loggauss(density_field, √2 , wave_vecs)

print("LogGauss Smoothing of 256")

 10.677264 seconds (8.88 M allocations: 1.008 GiB)
LogGauss Smoothing of 256

In [7]:
print("Fast Hessian Computed: ")
@time Hessian_256_fast = CosmoMMF.fast_hessian_from_smoothed(smoothed_256, √2 , wave_vecs)
print("\n")

print("Slow Hessian Computed: ")
@time Hessian_256_slow = CosmoMMF.slow_hessian_from_smoothed(smoothed_256, √2 , wave_vecs)
print("\n")


print("Calculations Completed")


Fast Hessian Computed:  15.518845 seconds (1.44 M allocations: 5.829 GiB, 3.31% gc time)

Slow Hessian Computed:  16.022347 seconds (1.37 M allocations: 7.317 GiB, 2.42% gc time)

Calculations Completed

In [8]:
if Hessian_256_fast == Hessian_256_slow
    print("They are Equivalent")
else
    print("They are not Equivalent")
end


They are not Equivalent

In [21]:
sum(Hessian_256_fast.^2)

2.245442035464335e16

In [14]:
Hessian_256_fast[1,1,1,3],Hessian_256_slow[1,1,1,3]

(-0.0, -0.18419653150756352)

### 512^3 Hessian Calculation

In [6]:
n = 512
wave_vecs = CosmoMMF.wavevectors3D(Float64,(n,n,n))
density_field = den_512_subset .+ 0.0001 
print("LogGauss Smoothing of 512: ")
@time smoothed_512 = CosmoMMF.smooth_loggauss(density_field, √2 , wave_vecs)

print("Finished Smoothing")

LogGauss Smoothing of 512:  59.321636 seconds (8.88 M allocations: 4.508 GiB, 0.48% gc time)
Finished Smoothing

In [ ]:
print("Fast Hessian Computed: ")
@time Hessian_512_fast = CosmoMMF.fast_hessian_from_smoothed(smoothed_512, √2 , wave_vecs)
print("\n")

print("Slow Hessian Computed: ")
@time Hessian_512_slow = CosmoMMF.slow_hessian_from_smoothed(smoothed_512, √2 , wave_vecs)
print("\n")


print("Calculations Completed")


In [ ]:
if Hessian_512_fast == Hessian_512_slow
    print("They are Equivalent")
else
    print("They are not Equivalent")
end